<a href="https://colab.research.google.com/github/Nischal5123/foreCache-interaction/blob/develop/StateActionProbab_ROI_Subset_Statistics_Mann_Kendall_Trend_ForeCache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk

In [ ]:
pip install pymannkendall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# this is implemented from STATE ACTION ANALYSIS FILE : https://colab.research.google.com/drive/1n9nJdd84etLey3piTKkRJmttpZlW1gor#scrollTo=bgGiHn-LobHm

#GET ALL STATE ACTION MODEL FOR NDSI 2D AND NDSI 3D
global_drive_path="/content/drive/MyDrive/ForeCache/interactions/stateActionModel/"
path=(str(global_drive_path) +"/ndsi-3d")
fileNames_NDSI_3D=[]
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filePath=(os.path.join(path, filename))
        fileNames_NDSI_3D.append(filename)
    else:
        continue
path=(str(global_drive_path) +"/ndsi-2d")
fileNames_NDSI_2D=[]
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filePath=(os.path.join(path, filename))
        fileNames_NDSI_2D.append(filename)
    else:
        continue
matchingName=[]
df_twoD=[]
df_threeD=[]
path_twoD=(str(global_drive_path) +"/ndsi-2d")
path_threeD=(str(global_drive_path) +"/ndsi-3d")
for i in range(len(fileNames_NDSI_2D)):
  for j in range(len(fileNames_NDSI_3D)):
  # if fileNames_NDSI_2D[i].removesuffix('taskname_ndsi-2d-task_') == fileNames_NDSI_3D[i].removesuffix('taskname_ndsi-3d-task_'):
   if fileNames_NDSI_2D[i][22:] == fileNames_NDSI_3D[j][22:]:
      #print(os.path.join(path, str(fileNames_NDSI_2D[i])))
      df=pd.read_csv(os.path.join(path_twoD, str(fileNames_NDSI_2D[i])),sep=',')
      df_twoD.append(df)
      df=pd.read_csv(os.path.join(path_threeD, str(fileNames_NDSI_3D[j])),sep=',')
      df_threeD.append(df)
      matchingName.append(fileNames_NDSI_2D[i])
#STORE ALL USERS NAME
users=[]
for name in matchingName:
    users.append(name[22:-4:1])

In [ ]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')
      
      self.pValue = adfTest[1]
      
      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False
      
      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)
    
    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries)
      
      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]
      
      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [ ]:
def make_numerical_dataframe(df):
  df['State_Action']=(df['Action']+ df['State'])
 
  df['Action']=pd.factorize(df['Action'])[0]
  df['State']=pd.factorize(df['State'])[0]
  state_action_correlated_series=pd.factorize(df['State_Action'])[0]
 
  action_only_series=df['Action'].values
  state_only_series=df['State'].values
  options=[state_action_correlated_series,action_only_series,state_only_series]
  return options

In [ ]:

#3D Mann Kendall
def Subset_screenshot_time(user,all_states_dataframe):
  failed_files=[]
  final_dataframe_3d = pd.DataFrame(columns=['User','StateNumber','Subtask_ROI','MKT Stationary_State_Action'])
  for i in tqdm(range(len(all_states_dataframe))):
    subtask=0
    length_parsedFile=0
    print("For state: ############",i)

    window_dataframe=all_states_dataframe[i]

    subtask_values=window_dataframe["Subtask_ROI"].unique()
    print(subtask_values)
    original_fileLength=len(window_dataframe)

    while ((length_parsedFile < original_fileLength) and (subtask <= len(subtask_values))):
      
      window_dataframe_subset=window_dataframe.loc[window_dataframe['Subtask_ROI'] == subtask_values[subtask]]
      length_parsedFile+=len(window_dataframe_subset)
      #print("Length of file",len(window_dataframe_subset), " in ","Subtask: ",subtask_values[subtask])
      #creating series
      state_action_correlated_series=window_dataframe_subset['StateActionProbab'].values

      three_series=[state_action_correlated_series]
      three_series_results=[user,i,subtask_values[subtask]]
      for series in (three_series):
        if len(series)>2:
          sTest_state = StationarityTests()
          sTest_state.Mann_Kendall_Stationarity_Test(series, printResults = False)
          three_series_results.append(sTest_state.trend)
          
        else:
          three_series_results.append("Not Enough Data")
          failed_files.append(user)
      #adding results per subtask per user to dataframe
      df_length = len(final_dataframe_3d)
      final_dataframe_3d.loc[df_length] = three_series_results
      subtask +=1
  return final_dataframe_3d
    

In [ ]:
#3D Mann Kendall
saveDir=(str(global_drive_path) +"StationarityTests/ndsi-3d/ActionBased")
combined_final_dataframe_3d = pd.DataFrame(columns=['User','StateNumber','Subtask_ROI','MKT Stationary_State_Action'])
array_of_dataframes=[]
for j in tqdm(range(len(df_threeD))):
  
  all_states_dataframes=[]
  # df_Navigation=df_threeD[j].loc[df_threeD[j]['State'] == 'Navigation']
  # all_states_dataframes.append(df_Navigation)
  df_Sensemaking=df_threeD[j].loc[df_threeD[j]['State'] == 'Sensemaking']
  all_states_dataframes.append(df_Sensemaking)
  # df_Answering=df_threeD[j].loc[df_threeD[j]['State'] == 'Answering']
  # all_states_dataframes.append(df_Answering)
  df_Foraging=df_threeD[j].loc[df_threeD[j]['State'] == 'Foraging']
  all_states_dataframes.append(df_Foraging)
  print(j)
  print(users[j])
  #running MKT on one user after dividing data based on states
  if (len(all_states_dataframes) == 2):
    actiondf=Subset_screenshot_time(users[j],all_states_dataframes)
    array_of_dataframes.append(actiondf)
    
  else:
    print("failed for user", user[j])
array_of_dataframes = pd.concat(array_of_dataframes)
userfilename= "All Users_ScreenShot_MKT_sliding_.xlsx"
array_of_dataframes.to_excel(os.path.join(saveDir,userfilename), index=False)
   

In [ ]:

#2D Mann Kendall
def Subset_screenshot_time(user,all_states_dataframe):
  failed_files=[]
  final_dataframe_2d = pd.DataFrame(columns=['User','StateNumber','Subtask_ROI','MKT Stationary_State_Action'])
  for i in tqdm(range(len(all_states_dataframe))):
    subtask=0
    length_parsedFile=0
    print("For state: ############",i)

    window_dataframe=all_states_dataframe[i]

    subtask_values=window_dataframe["Subtask_ROI"].unique()
    print(subtask_values)
    original_fileLength=len(window_dataframe)

    while ((length_parsedFile < original_fileLength) and (subtask <= len(subtask_values))):
      
      window_dataframe_subset=window_dataframe.loc[window_dataframe['Subtask_ROI'] == subtask_values[subtask]]
      length_parsedFile+=len(window_dataframe_subset)
      #print("Length of file",len(window_dataframe_subset), " in ","Subtask: ",subtask_values[subtask])
      #creating series
      state_action_correlated_series=window_dataframe_subset['StateActionProbab'].values

      three_series=[state_action_correlated_series]
      three_series_results=[user,i,subtask_values[subtask]]
      for series in (three_series):
        if len(series)>2:
          sTest_state = StationarityTests()
          sTest_state.Mann_Kendall_Stationarity_Test(series, printResults = False)
          three_series_results.append(sTest_state.trend)
          
        else:
          three_series_results.append("Not Enough Data")
          failed_files.append(user)
      #adding results per subtask per user to dataframe
      df_length = len(final_dataframe_2d)
      final_dataframe_2d.loc[df_length] = three_series_results
      subtask +=1
  return final_dataframe_2d
    

In [ ]:
#2D Mann Kendall
saveDir=(str(global_drive_path) +"StationarityTests/ndsi-2d/ActionBased")
combined_final_dataframe_3d = pd.DataFrame(columns=['User','StateNumber','Subtask_ROI','MKT Stationary_State_Action'])
array_of_dataframes=[]
for j in tqdm(range(len(df_threeD))):
  
  all_states_dataframes=[]
  df_Navigation=df_twoD[j].loc[df_twoD[j]['State'] == 'Navigation']
  all_states_dataframes.append(df_Navigation)
  df_Sensemaking=df_twoD[j].loc[df_twoD[j]['State'] == 'Sensemaking']
  all_states_dataframes.append(df_Sensemaking)
  df_Answering=df_twoD[j].loc[df_twoD[j]['State'] == 'Answering']
  all_states_dataframes.append(df_Answering)
  df_Foraging=df_twoD[j].loc[df_twoD[j]['State'] == 'Foraging']
  all_states_dataframes.append(df_Foraging)
  print(j)
  print(users[j])
  #running MKT on one user after dividing data based on states
  if (len(all_states_dataframes) == 4):
    actiondf=Subset_screenshot_time(users[j],all_states_dataframes)
    array_of_dataframes.append(actiondf)
    
  else:
    print("failed for user", user[j])
array_of_dataframes = pd.concat(array_of_dataframes)
userfilename= "All_Users_ScreenShot_MKT_sliding_.xlsx"
array_of_dataframes.to_excel(os.path.join(saveDir,userfilename), index=False)
   

  0%|          | 0/19 [00:00<?, ?it/s]

0
userid_82316e37-1117-4663-84b4-ddb6455c83b2



  5%|▌         | 1/19 [00:00<00:02,  6.51it/s]

For state: ############ 0
[1 2 3]
For state: ############ 1
[1 2 3]
For state: ############ 2
[]
For state: ############ 3
[1 2 3]
1
userid_d6555293-35c7-4c27-b522-38d67d2d1b1a



  0%|          | 0/4 [00:00<?, ?it/s]

...

 95%|█████████▍| 18/19 [00:01<00:00,  9.03it/s]

18
userid_93be0a02-3f7a-46c9-b526-aa956ed00856



  0%|          | 0/4 [00:00<?, ?it/s]

For state: ############ 0
[1 2 3]
For state: ############ 1
[1 2 3]
For state: ############ 2
[]
For state: ############ 3
[1 2 3]


100%|██████████| 19/19 [00:02<00:00,  9.22it/s]
